# Forte Tutorial 1.03: Forte's support for handling sparse states and operators

Forte exposes several functions to create and manipulate general second quantized operators and wave functions.
In this tutorial we will look at simple examples that illustrate how these classes work.

## Preliminaries
Here we define a useful function to display equations in LaTeX format

In [1]:
import math
import forte
from IPython.display import display, Math, Latex
from forte import det, SparseState, SparseOperator

def latex(obj):
    """Call the latex() function on an object and display the returned value in LaTeX"""
    display(Math(obj.latex()))

In a previous tutorial we looked at how to define determinants in forte. Here we are going to use the utility function `forte.det()`, which creates a determinant from a string representation of the determinant. The occupation of each orbital is specified by the symbols `2` (doubly occupied), `+` (single alpha electron), `-` (single beta electron), `0` (empty).

Here are some examples.

In [2]:
print(det('22+-'))
print(det('22ba'))
print(det('ABBA'))

|22+-000000000000000000000000000000000000000000000000000000000000>
|22-+000000000000000000000000000000000000000000000000000000000000>
|+--+000000000000000000000000000000000000000000000000000000000000>


Depending on the size if the `Determinant` class, these commands will return a 64 bit or longer representation of the determinants.

## The SparseState class

Sparse collections of determinants can be manipulated using the `SparseState` class. The simplest way to create a `SparseState` object is by passing a dictionary of `determinants -> double`. For example, here we create a superposition of a determinant with two electrons and one that has no electrons, with complex coefficients normalized to one:
$$
|\Psi\rangle = \frac{1}{\sqrt{2}} |20\rangle + \frac{i}{\sqrt{2}}|00\rangle 
$$

In [3]:
c = 1./ math.sqrt(2.0)
psi = SparseState({ forte.det('20'): c, forte.det('00') : c * 1j})
print(psi)

|2000000000000000000000000000000000000000000000000000000000000000> * (0.70710678,0.00000000)
|0000000000000000000000000000000000000000000000000000000000000000> * (0.00000000,0.70710678)



An alternative way to print this wave function is by calling the `str` method on the `SparseState` object. The argument `2` here indicates that we want to show only the occupation numbers of only the first two orbitals.

In [4]:
print(psi.str(2))

|20> * (0.70710678,0.00000000)
|00> * (0.00000000,0.70710678)



## The `SparseOperator` class

The `SparseOperator` class can handle operators of the form
$$
\hat{O} = \sum_{pqrs\cdots} t_{pq\cdots}^{rs\cdots} \hat{a}^\dagger_p \hat{a}^\dagger_q \cdots \hat{a}_s \hat{a}_r
$$
where each individual term in the summation can be an arbitrary order operator.
However, the amplitudes are assumed to be **real numbers**.

### Creating `SparseOperator` objects

After creation, a `SparseOperator` object is empty
```python
op = forte.SparseOperator()
latex(op)
# displays nothing
```

The simplest way to populate a `SparseOperator` is by adding one term at a time using the `add` function.

A generic operator
$$
\hat{q}_1 \hat{q}_2 \cdots, \quad \text{ with } \hat{q}_i \in \{ \hat{a}_p, \hat{a}^\dagger_p\}
$$
can be specified using the following syntax
```
add('[<orbital_1><spin_1><type_1> <orbital_2><spin_2><type_2> ...]', amplitude)
```
where
```
orbital_i: int
spin_i: 'a' (alpha) or 'b' (beta)
type_i: '+' (creation) or '-' (annihilation)
```

For example, the operator $-2 \hat{a}^\dagger_{1_\alpha} \hat{a}^\dagger_{2_\beta} \hat{a}_{3_\beta} \hat{a}_{0_\alpha}$ is encoded as `[1a+ 2b+ 3b- 0a-]` and can be generated with the following code:

In [5]:
op = forte.SparseOperator()
op.add('[1a+ 2b+ 3b- 0a-]',-2.0)
latex(op)

<IPython.core.display.Math object>

We can also directly build an operator using the `sparse_operator` convenience fuction, which uses the same syntax of `add` but allows to pass one or more operators as a list of pairs. For example, if we want to build the operator $\frac{1}{2} (\hat{a}_{0_\alpha} - \hat{a}^\dagger_{0_\alpha})$ we can do the following:

In [6]:
op = forte.sparse_operator([('[0a-]',0.5),('[0a+]',-0.5)])
# same as 
# op = forte.SparseOperator()
# op.add('[0a-]',0.5)
# op.add('[0a+]',-0.5)

latex(op)

<IPython.core.display.Math object>

Note that `SparseOperator` objects are addressable by operator string:

In [7]:
print(f"{op['[0a-]'] = }")
print(f"{op['[0a+]'] = }")
print(f"{op['[]'] = }")

op['[0a-]'] = (0.5+0j)
op['[0a+]'] = (-0.5+0j)
op['[]'] = 0j


They are also iterable, returning a pair of `SQOperatorString` and a coefficient

In [8]:
for sqop, c in op:
    print(sqop, c)

[0a+] (-0.5+0j)
[0a-] (0.5+0j)


### Ordering of second quantized operators in the `SparseOperator` object

<div class="alert alert-block alert-warning">
Note that <code>add</code> <b>assumes that the operators will match a specific order!</b>

This canonical order is defined as
$$
(\alpha \text{ creation}) (\beta \text{ creation}) (\beta \text{ annihilation}) (\alpha \text{ annihilation})
$$
with the creation (annihilation) operators ordered within each group in increasing (decreasing) order.
The following operator satisfies the canonical order:
$$
+\;\hat{a}_{2 \alpha}^\dagger\hat{a}_{3 \alpha}^\dagger\hat{a}_{2 \beta}^\dagger\hat{a}_{3 \beta}^\dagger\hat{a}_{1 \beta}\hat{a}_{0 \beta}\hat{a}_{1 \alpha}\hat{a}_{0 \alpha}
$$
</div>

If you want to work with operators that do not follow this ordering, for example, $\hat{a}_{1 \alpha}\hat{a}^\dagger_{0 \alpha}$, you will need to work out an equivalent representation, for example, $\hat{a}_{0 \alpha}\hat{a}^\dagger_{0 \alpha} = 1 - \hat{a}^\dagger_{0 \alpha}\hat{a}_{0 \alpha}$.

These examples illustrate valid operators in canonical order

In [9]:
# beta annihilation operators appear to the left of alpha annihilation
# within each group, orbital indices decrease going from left to right
op = forte.sparse_operator('[1b- 0b- 1a- 0a-]',1.0)
latex(op)

# beta creation operators appear to the right of alpha annihilation
# within each group, orbitals increase going from left to right
op = forte.sparse_operator('[0a+ 1a+ 0b+ 1b+]',1.0)
latex(op)

# creation operators appear to the left of annihilation operators
op = forte.sparse_operator('[2a+ 3a+ 2b+ 3b+ 1b- 0b- 1a- 0a-]',1.0)
latex(op)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

When the operator passed is out of order, an exception is thrown. For example, the following code
```python
op = forte.sparse_operator('[0b- 1b- 1a- 0a-]',1.0)
latex(op)
```
leads to the following RuntimeError

In [10]:
try:
    op = forte.sparse_operator('[0b- 1b- 1a- 0a-]',1.0)
except Exception as e:
    print(f'RuntimeError: {e}')

RuntimeError: Trying to initialize a SQOperator object with a product of
operators that are not arranged in the canonical form

    a+_p1 a+_p2 ...  a+_P1 a+_P2 ...   ... a-_Q2 a-_Q1   ... a-_q2 a-_q1
    alpha creation   beta creation    beta annihilation  alpha annihilation

with indices sorted as

    (p1 < p2 < ...) (P1 < P2 < ...)  (... > Q2 > Q1) (... > q2 > q1)
The operators are: 0a- 1a- 1b- 0b- 


This error can be overriden. However, **this is recommended only if you understand what happens when you do so**. The function `add_term_from_str` has an extra option that allows it to reorder the operators to the canonical form. The final operator is multiplied by a sign factor that corresponds to the parity of the permutation that connects the initial and final ordering. This code illustrates how this reordering happens

In [11]:
# the operators [0a- 0b- 1a- 1b-] are reordered and the final sign is -1. 
op = forte.sparse_operator('[0a- 0b- 1a- 1b-]',1.0,allow_reordering=True)
latex(op)

# the operators [0a- 0b- 1a- 1b-] are reordered and the final sign is -1. 
op = forte.sparse_operator('[0a- 0b- 1a- 1b-]',1.0,allow_reordering=True)
latex(op)

# The operator [0a- 0b- 1a- 1b-] (see above) is equivalent to -[1a- 1b- 0b- 0a-].
op = forte.sparse_operator('[1a- 1b- 0b- 0a-]',-1.0,allow_reordering=True)
latex(op)

# Another example that illustrates the reordering of operators
op = forte.sparse_operator('[0a- 0b- 1a- 1b- 2a+ 2b+ 3a+ 3b+]',1.0,allow_reordering=True)
latex(op)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

An exception is also thrown if two operators are repeated. For example, the following code
```python
op = forte.sparse_operator('[0b- 0b-]',1.0)
```
gives to the following RuntimeError

In [12]:
try:
    op = forte.sparse_operator('[0b- 0b-]',1.0)
except Exception as e:
    print(f'RuntimeError: {e}')

RuntimeError: Trying to initialize a SQOperator object with a product of
operators that contains repeated operators.



### Specifying a full operator with the `SparseOperator` class

To form a full operator we can just keep adding terms to a `SparseOperator` object or directly create a `SpareOperator` object by passing a list of pairs (string,coefficient) to the function `sparse_operator`. For example:

In [13]:
op1 = forte.SparseOperator()
op1.add('[1a+ 0a-]',0.3)
op1.add('[1b+ 0b-]',0.3)
op1.add('[1a+ 1b+ 0b- 0a-]',0.1)
latex(op1)

op2 = forte.sparse_operator([('[1a+ 0a-]',0.3),
                            ('[1b+ 0b-]',0.3),
                            ('[1a+ 1b+ 0b- 0a-]',0.1)])
latex(op2)

assert op1 == op2

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Applying a `SparseOperator` to a `SparseState`

To apply an operator to a state vector you can use the `apply` function of the `SparseState` class (or the function `forte.apply_op(op,psi)`). This function takes an operator (`op`) and applies it to the current state (`psi`), returning the state `|new_psi> = op |psi>`. For example, the following creates a CIS wave function using the operator
$$
\hat{T} = 0.1\; +0.3 \left(\hat{a}_{1 \alpha}^\dagger\hat{a}_{0 \alpha} + \hat{a}_{1 \beta}^\dagger\hat{a}_{0 \beta} \right)
$$
where the first term is just a scalar

In [14]:
op = forte.SparseOperator()
op.add('[]',0.1)
op.add('[1a+ 0a-]',0.3)
op.add('[1b+ 0b-]',0.3)
psi = forte.SparseState({ forte.det('2'): 1.0})
new_psi = psi.apply(op)
print(new_psi.str(3))

# test the apply_op function
assert new_psi[forte.det('2')] == 0.1
assert new_psi[forte.det('-+')] == 0.3
assert new_psi[forte.det('+-')] == 0.3

|+-0> * (0.30000000,0.00000000)
|-+0> * (0.30000000,0.00000000)
|200> * (0.10000000,0.00000000)



### Exponential operator

To apply the exponential operator $\exp(\hat{T})$ we can use the class `SparseExp` class. This class provides the method `apply_op` which takes as arguments the operator and the state

In [15]:
psi = forte.SparseState({ forte.det('2'): 1.0})
exp_op = forte.SparseExp()
new_psi = exp_op.apply_op(op,psi)
print(new_psi.str(3))

assert abs(new_psi[forte.det('200')]-1.105171) < 1e-6
assert abs(new_psi[forte.det('020')]-0.099465) < 1e-6
assert abs(new_psi[forte.det('+-0')]-0.331551) < 1e-6
assert abs(new_psi[forte.det('-+0')]-0.331551) < 1e-6

|020> * (0.09946538,0.00000000)
|-+0> * (0.33155128,0.00000000)
|+-0> * (0.33155128,0.00000000)
|200> * (1.10517092,0.00000000)



There are several variables that control the behavior of `apply`. For example, to compute the inverse, we can just apply $\exp(-\hat{T})$

In [16]:
new_psi2 = exp_op.apply_op(op,new_psi,scaling_factor=-1.0)
print(new_psi2.str(3))

|200> * (1.00000000,0.00000000)



We can also control other parameters, like the order of the Taylor expansion used to approximate $\exp(\hat{T})$ (`maxk`) and a threshold used to screen term (`screen_thresh`). For example, to apply $1 + \hat{T}$ we can call

In [17]:
psi = forte.SparseState({ forte.det('2'): 1.0})
exp_op = forte.SparseExp(maxk=1)
new_psi = exp_op.apply_op(op,psi)
print(new_psi.str(3))

|-+0> * (0.30000000,0.00000000)
|+-0> * (0.30000000,0.00000000)
|200> * (1.10000000,0.00000000)



### Exponential of an anti-Hermitian operator

The exponential function can also be used to compute the action of the anti-Hermitian operator $\hat{T} - \hat{T}^\dagger$. This operation is implemented in the class `SparseExp` via the method `apply_antiherm` which takes as arguments the operator and the state. Here is a simple example:

In [18]:
psi = forte.SparseState({ forte.det('2'): 1.0})
exp_op = forte.SparseExp()
new_psi = exp_op.apply_antiherm(op,psi)
print(new_psi.str(3))
norm = forte.overlap(new_psi,new_psi)
assert abs(norm-1.0) < 1e-6
assert abs(new_psi[forte.det('200')]-0.912668) < 1e-6
assert abs(new_psi[forte.det('020')]-0.087332) < 1e-6
assert abs(new_psi[forte.det('+-0')]-0.282321) < 1e-6
assert abs(new_psi[forte.det('-+0')]-0.282321) < 1e-6

|020> * (0.08733219,0.00000000)
|-+0> * (0.28232124,0.00000000)
|+-0> * (0.28232124,0.00000000)
|200> * (0.91266781,0.00000000)



### Factorized exponential of an anti-Hermitian operator

Another useful operator is the factorized exponential. Given a **list** of operators
$$
(t_1 \hat{\kappa}_1, t_2 \hat{\tau}_2, \ldots,t_N \hat{\tau}_N)
$$
the factorized exponential is defined as
$$
\prod_\mu^N \exp(t_\mu \hat{\tau}_\mu) = \exp(t_N \hat{\tau}_N) \cdots \exp(t_1 \hat{\tau}_1)  
$$
This operation is implemented in the class `SparseFactExp`.
This class provides the method `apply_antiherm` which takes as arguments the operator and the state. Here is a simple example:

In [19]:
op = forte.SparseOperatorList()
op.add('[1a+ 0a-]',0.3)
op.add('[1b+ 0b-]',0.3)

psi = forte.SparseState({ forte.det('2'): 1.0})
factexp_op = forte.SparseFactExp()
new_psi = factexp_op.apply_antiherm(op,psi)
print(new_psi.str(3))

# test the apply_antiherm function
assert abs(new_psi[forte.det('200')]-0.912668) < 1e-6
assert abs(new_psi[forte.det('020')]-0.087332) < 1e-6
assert abs(new_psi[forte.det('+-0')]-0.282321) < 1e-6
assert abs(new_psi[forte.det('-+0')]-0.282321) < 1e-6

|+-0> * (0.28232124,0.00000000)
|020> * (0.08733219,0.00000000)
|-+0> * (0.28232124,0.00000000)
|200> * (0.91266781,0.00000000)



To compute the inverse of the factorized exponential, just pass the option `inverse=True` to `apply_antiherm`:

In [20]:
starting_psi = factexp_op.apply_antiherm(op,new_psi,inverse=True)
print(starting_psi.str(3))

|200> * (1.00000000,0.00000000)



## Operator manipulation (New!)

### Algebraic operations

Forte allows the user to multiply and add many-body operators represented with the `SparseOperator` class. This allows the evaluation of operator expressions.

Operators can be scaled:

In [21]:
A = forte.sparse_operator('[1a+ 0a-]',1.0)
B = forte.sparse_operator('[1b+ 0b-]',1.0)

A *= 1.0 + 0.3j
print("A *= 0.3j:")
latex(A)

A *= 0.3j:


<IPython.core.display.Math object>

Two operators can be added or subtracted:

In [22]:
A = forte.sparse_operator('[1a+ 0a-]',1.0)
B = forte.sparse_operator('[1b+ 0b-]',1.0)
A_plus_B = A + B
print("A + B:")
latex(A_plus_B)

A_minus_B = A - B   
print("A - B:")
latex(A_minus_B)

A + B:


<IPython.core.display.Math object>

A - B:


<IPython.core.display.Math object>

Two operators can be multiplied:

In [23]:
A = forte.sparse_operator('[2a+ 1a-]',1.0)
B = forte.sparse_operator('[3a+ 0a-]',1.0)
AB = A * B
print("A x B:")
latex(AB)

A *= B
print("A *= B:")
latex(A)

A x B:


<IPython.core.display.Math object>

A *= B:


<IPython.core.display.Math object>

Note that the operators are returned in normal ordered form (with respect to the true vacuum).
For example, in the following case a contraction is possible among the operators:

In [24]:
A = forte.sparse_operator('[2a+ 3a-]',1.0)
B = forte.sparse_operator('[3a+ 0a-]',1.0)
AB = A * B
print("A x B:")
latex(AB)

A x B:


<IPython.core.display.Math object>

### Similarity transformations of operators

Forte can also perform similarity transformations of Hamiltonians (see [arXiv:2408.09636](https://arxiv.org/abs/2408.09636)). For example, the factorized unitary transformation of a `SparseOperator` $O$ by a sequence of exponentials:
$$
\bar{O} = \cdots \exp(-t_2 \hat{\tau}_2) \exp(-t_1 \hat{\tau}_1)  O \exp(t_1 \hat{\tau}_1) \exp(t_2 \hat{\tau}_2) \cdots  
$$
can be evaluated with the function `fact_unitary_trans_antiherm`

In [25]:
O = forte.sparse_operator('[1a+ 0a-]',1.0)
print(f'Untransformed operator = {O}')
T = forte.operator_list([('[2a+ 0a-]',0.3),('[1a+ 0a-]',0.5)])
print(f'Transformation generator = {T}')

Untransformed operator = (1 + 0i) * [1a+ 0a-]
Transformation generator = (0.3 + 0i) * [2a+ 0a-]
(0.5 + 0i) * [1a+ 0a-]


In [26]:
Obar = O.fact_unitary_trans_antiherm(T)
print(f'Transformed operator = {Obar}')

assert Obar['[1a+ 2a-]'] == -0.2593433800522308
assert Obar['[1a+ 0a-]'] == 0.735753498540072

Transformed operator = (-0.1416799342470381 + 0i) * [0a+ 2a-]
(-0.21958299058553404 + 0i) * [0a+ 1a-]
(-0.2593433800522308 + 0i) * [1a+ 2a-]
(-0.40194396816372097 + 0i) * [1a+ 1a-]
(0.40194396816372097 + 0i) * [0a+ 0a-]
(0.735753498540072 + 0i) * [1a+ 0a-]


It is also possible to evaluate the gradient of a transformation:
$$
\frac{\partial}{\partial t_k} \bar{O} = \frac{\partial}{\partial t_k} \cdots \exp(-t_2 \hat{\tau}_2) \exp(-t_1 \hat{\tau}_1)  O \exp(t_1 \hat{\tau}_1) \exp(t_2 \hat{\tau}_2) \cdots  
$$
This quantity can be evaluated with the function `fact_unitary_trans_antiherm_grad`


In [27]:
Obar_grad = O.fact_unitary_trans_antiherm_grad(T,0)
print(f'Transformed operator = {Obar_grad}')

assert Obar_grad['[0a+ 0a-]'] == -0.12433583966497525
assert Obar_grad['[0a+ 1a-]'] == 0.0679249787857943

Transformed operator = (-0.12433583966497525 + 0i) * [0a+ 0a-]
(-0.22759522787554526 + 0i) * [1a+ 0a-]
(-0.45801271084729195 + 0i) * [0a+ 2a-]
(-0.8383866435942036 + 0i) * [1a+ 2a-]
(0.0679249787857943 + 0i) * [0a+ 1a-]
(0.12433583966497525 + 0i) * [1a+ 1a-]


And we can easily verify that the computed gradients agree with numerical ones:

In [28]:
h = 1e-6
for i in range(2):
    Obar_grad = O.fact_unitary_trans_antiherm_grad(T,i)
    print(f'Analytical gradient = {Obar_grad}')

    # Two-point symmetric finite difference gradient
    Tp = forte.SparseOperatorList(T)
    Tp[i] = T[i] + h
    Tm = forte.SparseOperatorList(T)
    Tm[i] = T[i] - h
    Obar_grad_num = (O.fact_unitary_trans_antiherm(Tp) - O.fact_unitary_trans_antiherm(Tm))/(2 * h)
    print(f'Numerical gradient = {Obar_grad_num}')

    print(f'Difference norm = {(Obar_grad - Obar_grad_num).norm()}')
    assert (Obar_grad - Obar_grad_num).norm() < 1e-6

Analytical gradient = (-0.12433583966497525 + 0i) * [0a+ 0a-]
(-0.22759522787554526 + 0i) * [1a+ 0a-]
(-0.45801271084729195 + 0i) * [0a+ 2a-]
(-0.8383866435942036 + 0i) * [1a+ 2a-]
(0.0679249787857943 + 0i) * [0a+ 1a-]
(0.12433583966497525 + 0i) * [1a+ 1a-]
Numerical gradient = (-0.12433583965187898 + 0i) * [0a+ 0a-]
(-0.2275952278307791 + 0i) * [1a+ 0a-]
(-0.4580127108411158 + 0i) * [0a+ 2a-]
(-0.8383866435812681 + 0i) * [1a+ 2a-]
(0.06792497878049808 + 0i) * [0a+ 1a-]
(0.12433583965187898 + 0i) * [1a+ 1a-]
Difference norm = 5.0799151798035313e-11
Analytical gradient = (-0.2593433800522308 + 0i) * [0a+ 2a-]
(-0.5161705079545379 + 0i) * [1a+ 1a-]
(-0.8038879363274419 + 0i) * [0a+ 1a-]
(-0.8038879363274419 + 0i) * [1a+ 0a-]
(0.1416799342470381 + 0i) * [1a+ 2a-]
(0.5161705079545379 + 0i) * [0a+ 0a-]
Numerical gradient = (-0.25934338004907076 + 0i) * [0a+ 2a-]
(-0.5161705079470558 + 0i) * [1a+ 1a-]
(-0.8038879363292395 + 0i) * [1a+ 0a-]
(-0.8038879363431173 + 0i) * [0a+ 1a-]
(0.1416799342